In [1]:
#code is the for adding CIK to SPACAlpha
import pandas as pd
from fuzzywuzzy import process, fuzz

C:\Users\msnin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Read the dataframes
spac_alpha_data = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\SPACAlphadata.xlsx", sheet_name="testing")
all_companies_data = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\SPACAlphadata.xlsx", sheet_name="all companies + cik")

In [3]:
def fuzzy_match_and_update(row):
    # Convert tickers to string
    spac_ticker = str(row['SPAC Ticker'])
    newco_ticker = str(row['Newco Ticker'])
    newco_warrant_ticker = str(row['Newco Warrant Ticker'])
    
    # Fuzzy match for SPAC Ticker/Newco/NewcoWarrant
    spac_ticker_match = process.extractOne(spac_ticker, all_companies_data['Ticker'].astype(str), scorer=fuzz.ratio)
    newco_ticker_match = process.extractOne(newco_ticker, all_companies_data['Ticker'].astype(str), scorer=fuzz.ratio)
    newco_warrant_ticker_match = process.extractOne(newco_warrant_ticker, all_companies_data['Ticker'].astype(str), scorer=fuzz.ratio)

    # Check matches and update 'CIK Ticker' column
    if spac_ticker_match[1] >= newco_ticker_match[1] and spac_ticker_match[1] >= newco_warrant_ticker_match[1]:
        row['CIK Ticker'] = all_companies_data[all_companies_data['Ticker'].astype(str) == spac_ticker_match[0]]['cik'].values[0] if not all_companies_data[all_companies_data['Ticker'].astype(str) == spac_ticker_match[0]].empty else None
    elif newco_ticker_match[1] >= spac_ticker_match[1] and newco_ticker_match[1] >= newco_warrant_ticker_match[1]:
        row['CIK Ticker'] = all_companies_data[all_companies_data['Ticker'].astype(str) == newco_ticker_match[0]]['cik'].values[0] if not all_companies_data[all_companies_data['Ticker'].astype(str) == newco_ticker_match[0]].empty else None
    elif newco_warrant_ticker_match[1] >= spac_ticker_match[1] and newco_warrant_ticker_match[1] >= newco_ticker_match[1]:
        row['CIK Ticker'] = all_companies_data[all_companies_data['Ticker'].astype(str) == newco_warrant_ticker_match[0]]['cik'].values[0] if not all_companies_data[all_companies_data['Ticker'].astype(str) == newco_warrant_ticker_match[0]].empty else None
    else:
        row['CIK Ticker'] = None

    return row


In [4]:
# Apply the fuzzy matching and updating function
spac_alpha_data = spac_alpha_data.apply(fuzzy_match_and_update, axis=1)


In [5]:
# Save the updated DataFrame to a new Excel file
spac_alpha_data.to_excel("updated_SPACAlphadata420.xlsx", index=False)
